In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import ipywidgets as widgets
import datetime

from IPython.display import display
from ipyfilechooser import FileChooser
from time import sleep
import thorlabs_apt as apt
import pyvisa

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default='notebook' # for vscode ,maybe 'colab' on jupyterlab 

In [ ]:
def apt_device_list():
    print(apt.list_available_devices())

def stage_move_to():
    global stage_serial_num
    global stage_target_pos
    motor = apt.Motor(stage_serial_num)
    motor.move_to(stage_target_pos, blocking=True)

def stage_velocity():
    global stage_serial_num
    global stage_max_velocity
    motor = apt.Motor(stage_serial_num)
    min_v, acc, max_v = motor.get_velocity_parameters()
    motor.set_velocity_parameters(min_v, acc, stage_max_velocity)

def stage_enable():
    global stage_serial_num
    motor = apt.Motor(stage_serial_num)
    motor.enable()

def stage_disable():
    global stage_serial_num
    motor = apt.Motor(stage_serial_num)
    motor.disable()

def start_DMM6500():
    global DMM6500_visa_address
    rm = pyvisa.ResourceManager()
    with rm.open_resource(DMM6500_visa_address) as instr:
        instr.write(':TRACe:CLEar')
        instr.write(':TRIGger:LOAD "LoopUntilEvent", COMMand, 100')
        instr.write('INIT')

def stop_DMM6500():
    global DMM6500_visa_address
    rm = pyvisa.ResourceManager()
    with rm.open_resource(DMM6500_visa_address) as instr:
        instr.write(':ABORT')
    
def read_DMM6500():
    global now
    now = datetime.datetime.now()
    global DMM6500_visa_address
    global x, y
    rm = pyvisa.ResourceManager()
    with rm.open_resource(DMM6500_visa_address) as instr:
        result = instr.query(':TRIGger:STATe?')
        assert 'ABORTED' in result.split(';')
        result = instr.query(':TRACe:ACTual:END?')
        end = int(result)
        assert end > 0
        result = instr.query(f':TRACe:DATA? 1,{end},"defbuffer1",RELative')
        x = np.array(list(map(float,result.split(','))))
        result = instr.query(f':TRACe:DATA? 1,{end},"defbuffer1"')
        y = np.array(list(map(float,result.split(','))))

def plot_xy():
    global x,y
    fig = make_subplots(rows=1, cols=1, subplot_titles=("x,y", "---somethingsomething"))
    go_element = lambda x,y,name: go.Scatter(x=x,y=y,mode='lines',name=name)
    fig.add_trace(go_element(x,y, 'x,y'), row=1, col=1)
    fig.update_xaxes(title_text="x", row=1, col=1)
    fig.update_yaxes(title_text="y", row=1, col=1)
    fig.show()

def save_xy():
    global comment
    global now
    filename_y = now.strftime('%Y%m%d_%H%M%S') + '_' + 'ydata_' + comment   + '.npy'
    filename_x = now.strftime('%Y%m%d_%H%M%S') + '_' +'xdata_' + comment +  '.npy'
    np.save(filename_y,y)
    np.save(filename_x,x)

        

In [ ]:


def simple_ui():
    button_clear_output = widgets.Button(description='表示クリア')
    button_apt_device_list = widgets.Button(description='apt_device_list')
    button_stage_enable = widgets.Button(description='stage_enable')
    button_stage_disable = widgets.Button(description='stage_disable')
    button_stage_move_to = widgets.Button(description='stage_move_to')
    button_stage_velocity = widgets.Button(description='stage_velocity')
    button_start_DMM6500 = widgets.Button(description='start_DMM6500')
    button_stop_DMM6500 = widgets.Button(description='stop_DMM6500')
    button_read_DMM6500 = widgets.Button(description='read_DMM6500')
    button_plot_xy = widgets.Button(description='plot_xy')
    button_save_xy = widgets.Button(description='save_xy')
    filechooser = FileChooser('./')
    button_load_as_ydata_npy = widgets.Button(description='ydata.npyとして読込')
    button_load_as_xdata = widgets.Button(description='xdata.npyとして読込')
    button_input_field = widgets.Button(description='強制変数反映',disabled=False)

    text_comment = widgets.Text(value='',placeholder='文字を入力',description='comment',disabled=False,style={'description_width': 'initial'})
    text_stage_serial_num = widgets.Text(value='28252436',placeholder='28252436',description='stage_serial_num',disabled=False, style={'description_width': 'initial'})
    float_stage_target_pos = widgets.FloatText(value=0,description='stage_target_pos',disabled=False,style={'description_width': 'initial'})
    float_stage_max_velocity = widgets.FloatText(value=99,description='stage_max_velocity',disabled=False,style={'description_width': 'initial'})
    text_DMM6500_visa_address = widgets.Text(value='TCPIP0::169.254.6.100::inst0::INSTR',placeholder='visa_address',description='DMM6500_visa_address',disabled=False, style={'description_width': 'initial'})


    output = widgets.Output(layour={'border': '1px solid black'})
    def wrapped_func_factory(func,verbose=True):
        def new_func(ui_element):
            with output:
                if verbose: print(f"[exec func {func.__name__}]")
                func()
                if verbose: print(f"[complete {func.__name__}]")
        return new_func
    button_clear_output.on_click(lambda button: output.clear_output(wait=False))

    def load_npy_factory(variable_name):
        def load_npy():
            choosed_file_path = filechooser.selected
            global x,y
            if variable_name == 'y':
                y = np.load(choosed_file_path)
                print(f'y.shape={y.shape}')
            elif variable_name == 'x':
                x = np.load(choosed_file_path)
                print(f'x.shape={x.shape}')
        return load_npy
        
    button_load_as_ydata_npy.on_click(wrapped_func_factory(load_npy_factory('y')))
    button_load_as_xdata.on_click(wrapped_func_factory(load_npy_factory('x')))
    
    def load_input_field():
        global comment
        global stage_serial_num
        global stage_target_pos
        global stage_max_velocity
        global DMM6500_visa_address
        comment = text_comment.value
        stage_serial_num = int(text_stage_serial_num.value)
        stage_target_pos = float_stage_target_pos.value
        stage_max_velocity = float_stage_max_velocity.value
        DMM6500_visa_address = text_DMM6500_visa_address.value
    button_input_field.on_click(wrapped_func_factory(load_input_field,verbose=False))
    button_apt_device_list.on_click(wrapped_func_factory(apt_device_list))
    button_stage_move_to.on_click(wrapped_func_factory(stage_move_to))
    button_stage_velocity.on_click(wrapped_func_factory(stage_velocity))
    button_stage_enable.on_click(wrapped_func_factory(stage_enable))
    button_stage_disable.on_click(wrapped_func_factory(stage_disable))
    button_start_DMM6500.on_click(wrapped_func_factory(start_DMM6500))
    button_stop_DMM6500.on_click(wrapped_func_factory(stop_DMM6500))
    button_read_DMM6500.on_click(wrapped_func_factory(read_DMM6500))
    button_plot_xy.on_click(wrapped_func_factory(plot_xy))
    button_save_xy.on_click(wrapped_func_factory(save_xy))
    
    input_fields = [text_comment, text_stage_serial_num, float_stage_target_pos, float_stage_max_velocity]
    input_fields+= [text_DMM6500_visa_address]
    for field in input_fields:
        field.continuous_update = False
        field.observe(wrapped_func_factory(load_input_field,verbose=False))

    display(
        widgets.VBox([
            widgets.HBox([button_clear_output]),
            widgets.HBox([button_apt_device_list,button_stage_disable, button_stage_enable,button_stage_move_to,button_stage_velocity]),
            widgets.HBox([button_start_DMM6500,button_stop_DMM6500,button_read_DMM6500]),
            widgets.HBox([button_plot_xy, button_save_xy]),
            widgets.HBox([filechooser, button_load_as_ydata_npy, button_load_as_xdata]),
            widgets.HBox([text_comment,  button_input_field]),
            widgets.HBox([text_stage_serial_num, float_stage_target_pos,float_stage_max_velocity]),
            widgets.HBox([text_DMM6500_visa_address]),
            output
        ])
    )


In [ ]:
simple_ui()